In [ ]:
## Install librariers
!pip install transformers

In [ ]:
## Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json


In [ ]:
## Import Model
model_name = "vinaybabu/Qwen3-1.7B-SharedTask-Finetuned-QA_V2-checkpoint-1500_v1"
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

In [ ]:
## Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = "/content/drive/MyDrive/nlpai4health-shared-task/test_data_release"   # add the test dataset location

In [ ]:
import os
list_of_files = ['Assamese']   # mention the language. I have run seperately for each language due to timeout issues

In [ ]:
system_prompt = """

You are a multilingual medical assistant specialized in interpreting dialogues between patients and health workers to answer questions accurately, safely, and in context.

You are given a dialogue and question in language **{}** Provide the answer in the same language

### GOAL
Given:
1. A multi-turn dialogue between a *health worker* and a *patient* (sometimes including relatives).
2. A user question referring to the conversation.

You must generate an **accurate, contextually faithful answer** to the question, **in the same language** as the dialogue and question.

---

### INSTRUCTIONS

1. **Read the dialogue carefully.**
   - Identify key facts, medical information, symptoms, lifestyle habits, diagnoses, and doctor advice.
   - Pay attention to *who said what* (Patient vs. Health Worker).

2. **Understand the question.**
   - Determine whether it is asking for an explanation, definition, reassurance, or instruction.
   - If it refers to a previous statement in the dialogue, infer the correct medical meaning.

3. **Generate a response that is:**
   - **Accurate** — consistent with the dialogue and medical reasoning.
   - **Clear and conversational** — natural tone as if the health worker is directly replying.
   - **Safe** — no medical advice beyond what is supported by the dialogue.
   - **In the same language** as the input (e.g., English, Hindi, Tamil, Telugu, etc.).
   - **Comprehensive but concise** — typically 2–5 sentences.

4. **DO NOT:**
   - Invent new facts, symptoms, or recommendations not mentioned or implied.
   - Translate or change the language.
   - Repeat the question verbatim unless necessary for clarity.

---

### OUTPUT FORMAT

**Answer:**
Provide only the direct answer text, without labels, metadata, or formatting markers. Provide atleast 2-3 lines of answer

"""

In [ ]:
i=0
output_folder_to_write = "/content/drive/MyDrive/nlpai4health-shared-task/test_data_release/Assamese/QnA_output/"   # Output folder location for writing the output files
available_output_files = os.listdir(output_folder_to_write)
available_output_files_modified = [
    fname.split('_Open')[1].split('_questions')[0]
    for fname in available_output_files
    if '_Open' in fname and '_questions' in fname
]
for language in list_of_files:
  dialogue_folder = "Dialogues"
  questions_folder = "QnA"
  list_of_files_folder1 = os.listdir(os.path.join(dataset_path, language, dialogue_folder))
  list_of_files_folder2 = os.listdir(os.path.join(dataset_path, language, questions_folder))
  list_of_files_folder1_without_extension = [file.split(".")[0] for file in list_of_files_folder1]
  list_of_files_folder2_without_extension = [file.split(".")[0] for file in list_of_files_folder2]
  print(list_of_files_folder2_without_extension)
  for file in list_of_files_folder1_without_extension:
    if file in available_output_files_modified:
      print("file is already available", file)
      continue
    else:
      file_to_check = file +"_questions_blank"
      print(i)
      i = i+1
      if file_to_check in list_of_files_folder2_without_extension:
          list_qa_pairs = []
          with open(os.path.join(dataset_path, language, dialogue_folder, file+".jsonl"), "r", encoding="utf-8") as f:
              dialogue = f.read()
          with open(os.path.join(dataset_path, language, questions_folder, file_to_check+".json"), "r", encoding="utf-8") as f:
              try:
                  questions = f.read()
              except:
                  print("the file is not found", file_to_check)
                  continue
              extracted_questions = json.loads(questions)
          #print(dialogue)
          # questions_list = extracted_questions["questions"]
          # list_qa_pairs = []
          # # --- BATCHED INFERENCE STARTS HERE ---
          # # make prompts for all questions first
          # def build_prompt(q):
          #   system_prompt_with_language = system_prompt.format(language)
          #   messages = [
          #       {"role": "system", "content": system_prompt_with_language},
          #       # include dialogue context if the model needs it to answer:
          #       {"role": "user", "content": f"{dialogue}\n\nQuestion: {q}"}
          #   ]
          #   # produce a single chat formatted string (no tokenization yet)
          #   return tokenizer.apply_chat_template(
          #       messages,
          #       tokenize=False,
          #       add_generation_prompt=True,
          #       enable_thinking=True,  # if your model supports it
          #   )

          # prompts = [build_prompt(q) for q in questions_list]

          # # chunk into batches
          # num_batches = math.ceil(len(prompts) / BATCH_SIZE)
          # for b in range(num_batches):
          #     batch_prompts = prompts[b*BATCH_SIZE : (b+1)*BATCH_SIZE]
          #     batch_questions = questions_list[b*BATCH_SIZE : (b+1)*BATCH_SIZE]

          #     # tokenize as a batch with left padding
          #     enc = tokenizer(
          #         batch_prompts,
          #         return_tensors="pt",
          #         padding=True,
          #         truncation=True,      # truncates long dialogue if needed
          #     ).to(device)

          #     with torch.inference_mode():
          #         gen_ids = model.generate(
          #             input_ids=enc.input_ids,
          #             attention_mask=enc.attention_mask,
          #             max_new_tokens=32768,
          #         )

          #     # slice off the prompt for each sample
          #     out_ids = [
          #         gen_ids[i, enc.input_ids.shape[1]:].tolist()
          #         for i in range(gen_ids.size(0))
          #     ]

          #     # optional: strip <think> content per sample
          #     for q, ids in zip(batch_questions, out_ids):
          #         try:
          #             # find last THINK_EOS_ID (</think>)
          #             cut = len(ids) - ids[::-1].index(THINK_EOS_ID)
          #         except ValueError:
          #             cut = 0

          #         thinking_text = tokenizer.decode(ids[:cut], skip_special_tokens=True).strip()
          #         answer_text   = tokenizer.decode(ids[cut:], skip_special_tokens=True).strip()

          #         list_qa_pairs.append({
          #             "question": q,
          #             "answer": answer_text,          # store just the answer
          #             "debug_thinking": thinking_text # optional: remove in prod
          #         })

          # # write once per file
          # print("writing_to_file")
          # out_name = f"Team KV_Q&A_Open{file_to_check}_QnA.json"
          # with open(out_name, "w", encoding="utf-8") as f:
          #     json.dump({"questions": list_qa_pairs}, f, ensure_ascii=False, indent=2)



          for each_question in extracted_questions['questions']:
            print(each_question)
            system_prompt_with_language = system_prompt.format(language)
            messages = [{"role":"system","content":system_prompt_with_language},{"role":"user","content":each_question["question"]}]
            text = tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True,enable_thinking=False) # Switches between thinking and non-thinking modes. Default is True.)
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
            # conduct text completion
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=32768
            )
            output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

            # parsing thinking content
            try:
                # rindex finding 151668 (</think>)
                index = len(output_ids) - output_ids[::-1].index(151668)
            except ValueError:
                index = 0

            thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
            content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
            print(content)
            list_qa_pairs.append({"question":each_question['question'],"answer":content})
          print("writing_to_file")
          file_to_write = "Team_KV"+"_Q&A"+"_Closed"+file_to_check+"_QnA"+".json"
          with open(output_folder_to_write+file_to_write, "w") as file:
            json.dump({"questions":list_qa_pairs}, file)


['scenario_54_0633f08ce93045468c427ed19d21ca3e_IDX_03_0_questions_blank', 'scenario_1_68b79d0db068477ebb9c2a90cd998751_IDX_04_0_questions_blank', 'scenario_98_c296c9b365bc4fa981713cf77a26513f_IDX_00_1_questions_blank', 'scenario_81_86e1b79ae75442d284e3a954504a052e_IDX_00_0_questions_blank', 'scenario_84_e9fd62d8c94644daadeccfd4d9af74d1_IDX_01_4_questions_blank', 'scenario_35_d2af26db00814bb4899f452fe7d7f104_IDX_00_0_questions_blank', 'scenario_37_087ebc8983ed49a3bb4e1646cc773069_IDX_02_0_questions_blank', 'scenario_67_5a07f004bc5f45cb8513c11e8e676ada_IDX_04_4_questions_blank', 'scenario_31_d950917610b34864a6ff0cf6d1287564_IDX_03_4_questions_blank', 'scenario_3_32d5c44653f14b26b599342a1df28e88_IDX_02_5_questions_blank', 'scenario_60_09a21a9d370049fa875a92105695dc98_IDX_00_1_questions_blank', 'scenario_24_530828fbd9c94bc385698bc95f990e81_IDX_03_3_questions_blank', 'scenario_35_cb6d5508dee445fa82b320b62492884f_IDX_02_0_questions_blank', 'scenario_2_c233e3aeff9545c9912ac81a4d8cfbc0_IDX_00_